1. 라이브러리 및 데이터

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [3]:
# load data
train = pd.read_csv('C:/Users/zz325/PycharmProjects/2020742043/Datamining/1.Training/원천데이터/2.수면/train_sleep.csv')
train_label = pd.read_csv('C:/Users/zz325/PycharmProjects/2020742043/Datamining/1.Training/라벨링데이터/2.수면/training_label.csv')
test = pd.read_csv('C:/Users/zz325/PycharmProjects/2020742043/Datamining/2.Validation/원천데이터/2.수면/val_sleep.csv')
test_label = pd.read_csv('C:/Users/zz325/PycharmProjects/2020742043/Datamining/2.Validation/라벨링데이터/2.수면/val_label.csv')


In [4]:
# train data 연구 참여자 수: 141 = 85(CN) + 47(MCI) + Dem(9)
# test data 연구 참여자 수: 33 = 26(CN) + MCI(4) + Dem(3)

print(train.shape)
print(train_label.shape)
print(test.shape)
print(test_label.shape)
print(train_label['DIAG_NM'].value_counts())
print(test_label['DIAG_NM'].value_counts())

(9705, 36)
(141, 2)
(2478, 36)
(33, 2)
DIAG_NM
CN     85
MCI    47
Dem     9
Name: count, dtype: int64
DIAG_NM
CN     26
MCI     4
Dem     3
Name: count, dtype: int64


In [5]:
def preprocessing(train, label):
    drop_cols = train.describe(include = 'O').columns[1:]
    train = train.drop(drop_cols, axis = 1)
    data = train.groupby('EMAIL').mean().reset_index()
    labeling = dict(np.array(label))

    data['target'] = data['EMAIL'].map(labeling)
    train['target'] = train['EMAIL'].map(labeling)

    # target 변수 numerical data로 변환
    target_dict = {"CN": 1, "MCI": 2, "Dem": 3}
    data['target'] = data['target'].map(target_dict)
    data = data.sort_values(by=['target','EMAIL'], ascending=[True,True])
    data = data.reset_index(drop=True)
    
    train['target'] = train['target'].map(target_dict)
    train = train.sort_values(by=['target','EMAIL'], ascending=[True,True])
    train = train.reset_index(drop=True)

    # Step 1: 'number' 열을 생성 (EMAIL에서 숫자만 추출)
    data['number'] = data['EMAIL'].str.extract(r'(?<=nia\+)(\d+)', expand=False).astype(int)
    train['number'] = train['EMAIL'].str.extract(r'(?<=nia\+)(\d+)', expand=False).astype(int)

    # Step 2: EMAIL 열 삭제
    data.drop(columns=['EMAIL'], inplace=True)
    train.drop(columns=['EMAIL'], inplace=True)

    return data, train

train_mean, train_all = preprocessing(train, train_label)
train_all

,sleep_awake,sleep_breath_average,sleep_deep,sleep_duration,sleep_efficiency,sleep_hr_average,sleep_hr_lowest,sleep_is_longest,sleep_light,sleep_midpoint_at_delta,...,sleep_score_disturbances,sleep_score_efficiency,sleep_score_latency,sleep_score_rem,sleep_score_total,sleep_temperature_delta,sleep_temperature_deviation,sleep_total,target,number
0,1860,15.875,5640,24480,92,59.63,54,1,11670,10776,...,79,97,94,72,61,-0.13,-0.13,22620,1,5
1,1860,16.000,9660,31200,94,60.74,53,1,11250,8204,...,80,99,99,99,88,0.11,0.11,29340,1,5
2,3210,15.625,7230,30420,89,61.25,53,1,14100,8574,...,79,95,97,79,79,-0.10,-0.10,27210,1,5
3,1890,16.250,6690,28980,93,60.26,53,1,15750,9217,...,81,99,91,63,79,0.13,0.13,27090,1,5
4,1200,15.875,4980,30180,96,60.71,55,1,18600,8349,...,84,100,72,73,86,-0.41,-0.41,28980,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9700,9480,15.625,3720,46200,79,66.78,60,1,20580,6719,...,56,72,94,100,100,0.41,0.41,36720,3,341
9701,8280,15.000,3480,38280,78,70.26,62,1,18030,3181,...,62,67,99,99,96,0.07,0.07,30000,3,341
9702,2580,17.625,4890,26940,90,69.55,63,1,13470,10501,...,60,96,72,90,74,-0.14,-0.14,24360,3,341
9703,8940,16.625,4470,40860,78,70.85,63,1,18420,5415,...,63,67,97,100,99,0.25,0.25,31920,3,341


In [6]:
print(len(train_all.columns))

29


In [7]:
def feature_by_pvalue(df):
    independent_vars = list(df.columns)

    anova_results = {}

    for var in independent_vars:
        group0 = df[df['target'] == 1][var]
        group1 = df[df['target'] == 2][var]
        group2 = df[df['target'] == 3][var]
    
        f_val, p_val = stats.f_oneway(group0, group1, group2)
        anova_results[var] = p_val

    sorted_vars = sorted(anova_results, key=anova_results.get)

    significant_vars = [var for var in sorted_vars if anova_results[var] < 0.05]

    print("유의미한 변수들:", significant_vars)
    print(len(significant_vars))

    return significant_vars

In [8]:
data_for_feature_all = train_all
feature_by_pvalue(data_for_feature_all)

유의미한 변수들: ['target', 'number', 'sleep_light', 'sleep_restless', 'sleep_midpoint_time', 'sleep_duration', 'sleep_total', 'sleep_score_deep', 'sleep_score_alignment', 'sleep_awake', 'sleep_deep', 'sleep_score_total', 'sleep_onset_latency', 'sleep_midpoint_at_delta', 'sleep_score_latency', 'sleep_breath_average', 'sleep_hr_lowest', 'sleep_score_efficiency', 'sleep_efficiency', 'sleep_rem', 'sleep_score_disturbances', 'sleep_hr_average', 'sleep_period_id', 'sleep_score_rem', 'sleep_rmssd', 'sleep_temperature_delta', 'sleep_temperature_deviation']
27


c:\anaconda3\envs\datamining\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  res = hypotest_fun_out(*samples, **kwds)


['target',
 'number',
 'sleep_light',
 'sleep_restless',
 'sleep_midpoint_time',
 'sleep_duration',
 'sleep_total',
 'sleep_score_deep',
 'sleep_score_alignment',
 'sleep_awake',
 'sleep_deep',
 'sleep_score_total',
 'sleep_onset_latency',
 'sleep_midpoint_at_delta',
 'sleep_score_latency',
 'sleep_breath_average',
 'sleep_hr_lowest',
 'sleep_score_efficiency',
 'sleep_efficiency',
 'sleep_rem',
 'sleep_score_disturbances',
 'sleep_hr_average',
 'sleep_period_id',
 'sleep_score_rem',
 'sleep_rmssd',
 'sleep_temperature_delta',
 'sleep_temperature_deviation']

In [9]:
data_for_feature_mean = train_mean
feature = feature_by_pvalue(data_for_feature_mean)

유의미한 변수들: ['target', 'number', 'sleep_light', 'sleep_score_latency', 'sleep_midpoint_time', 'sleep_duration', 'sleep_onset_latency', 'sleep_restless', 'sleep_score_alignment', 'sleep_total', 'sleep_score_deep', 'sleep_awake']
12


모델적용

In [10]:
X_mean = train_mean
X_mean.drop(columns=['number'], inplace=True)
X_mean_target = X_mean['target']
X_mean.drop(columns=['target'], inplace=True)

In [11]:
print(X_mean.values.shape)

(141, 27)


In [12]:
# significant value list에 number와 target이 포함되어 있어 삭제
feature.remove('number')
feature.remove('target')

In [13]:
data_for_logit = X_mean[feature].copy()
data_for_logit


,sleep_light,sleep_score_latency,sleep_midpoint_time,sleep_duration,sleep_onset_latency,sleep_restless,sleep_score_alignment,sleep_total,sleep_score_deep,sleep_awake
0,16303.539823,83.973451,15127.699115,30115.752212,672.212389,29.300885,86.849558,28060.088496,89.345133,2055.663717
1,14088.000000,74.800000,14404.285714,28422.857143,578.571429,33.571429,89.028571,23275.714286,96.542857,5147.142857
2,12606.891892,79.905405,13722.972973,27224.594595,528.243243,32.202703,89.500000,21789.324324,99.081081,5435.270270
3,14666.202532,84.000000,17314.556962,34593.417722,817.974684,29.696203,95.430380,25923.037975,92.063291,8670.379747
4,18325.890411,83.945205,14871.780822,30269.589041,866.712329,43.602740,99.616438,24827.671233,96.904110,5441.917808
...,...,...,...,...,...,...,...,...,...,...
136,14092.500000,81.931818,14036.590909,26896.363636,991.363636,51.931818,89.704545,19431.136364,56.909091,7465.227273
137,19578.620690,68.609195,18986.896552,36944.827586,1412.413793,27.850575,99.942529,32936.206897,96.919540,4008.620690
138,20261.756757,72.878378,15657.972973,31551.081081,459.729730,46.729730,89.216216,27824.189189,48.081081,3726.891892
139,14174.262295,81.983607,12480.491803,24405.245902,641.311475,41.360656,97.737705,21195.245902,94.377049,3210.000000


In [14]:
# 다중 로지스틱 회귀 구현
import torch
import torch.nn as nn

class DementiaNet(nn.Module):
    def __init__(self):
        super(DementiaNet, self).__init__()
        self.fc1 = nn.Linear(10, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 3)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(data_for_logit)

x = torch.tensor(X_scaled).float()
y = torch.tensor(X_mean_target.values - 1).long()

In [15]:
net = DementiaNet()
opt = torch.optim.Adam(net.parameters(), lr=1e-3)
los = nn.CrossEntropyLoss()

In [ ]:
from collections import defaultdict
 
bs = 16
epoch = 20

for e in range(1,epoch + 1):
    eloss = defaultdict(lambda: [])
    for idx in range(0,len(x),bs):
        opt.zero_grad()
        p = net(x[idx:idx+bs])
        loss = los(p, y[idx:idx+bs])
        loss.backward()
        opt.step()
        # keep track of some summary statistics
        eloss['loss'].append(loss.item())
        eloss['n'].append(x[idx:idx+bs].size(0))
        eloss['corr'].append(sum(p.max(axis=1)[1] == y[idx:idx+bs]))
    print(f'epoch {e:3d} avg loss; {sum(eloss["loss"])/sum(eloss["n"]):.4f}')
    print(f'        acc.;    {sum(eloss["corr"])/sum(eloss["n"]):.4f}')
print("done")

net.eval()
with torch.no_grad():
    final_accuracy = (net(x).argmax(dim=1) == y).sum().item() / len(y)
print(f"Final accuracy: {final_accuracy:.4f}")

epoch   1 avg loss; 0.0743
        acc.;    0.2624
epoch   2 avg loss; 0.0635
        acc.;    0.6028
epoch   3 avg loss; 0.0598
        acc.;    0.6028
epoch   4 avg loss; 0.0580
        acc.;    0.6028
epoch   5 avg loss; 0.0569
        acc.;    0.6028
epoch   6 avg loss; 0.0563
        acc.;    0.6028
epoch   7 avg loss; 0.0559
        acc.;    0.6028
epoch   8 avg loss; 0.0555
        acc.;    0.6028
epoch   9 avg loss; 0.0553
        acc.;    0.6028
epoch  10 avg loss; 0.0550
        acc.;    0.6028
epoch  11 avg loss; 0.0547
        acc.;    0.6028
epoch  12 avg loss; 0.0545
        acc.;    0.6028
epoch  13 avg loss; 0.0542
        acc.;    0.6028
epoch  14 avg loss; 0.0539
        acc.;    0.6028
epoch  15 avg loss; 0.0537
        acc.;    0.6028
epoch  16 avg loss; 0.0534
        acc.;    0.6028
epoch  17 avg loss; 0.0532
        acc.;    0.6028
epoch  18 avg loss; 0.0530
        acc.;    0.6028
epoch  19 avg loss; 0.0528
        acc.;    0.6028
epoch  20 avg loss; 0.0526
    

In [17]:
_, test_mean = preprocessing(test, test_label)

tmp_mean_target = test_mean['target']

tmp_test = test_mean[feature].copy()

test_X_scaled = scaler.transform(tmp_test)
test_x = torch.tensor(test_X_scaled).float()
test_y = torch.tensor(tmp_mean_target.values - 1).long()

net.eval()

with torch.no_grad():
    predictions = net(test_x)
    predicted_labels = predictions.max(dim=1)[1]
    
    accuracy = (predicted_labels == test_y).sum().item() / len(test_y)
    print(f'Test Accuracy: {accuracy:.4f}')

    test_loss = los(predictions, test_y)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Accuracy: 0.7910
Test Loss: 0.7390


In [18]:
X_all = train_all
X_all.drop(columns=['number'], inplace=True)
X_all_target = X_all['target']
X_all.drop(columns=['target'], inplace=True)

print(X_all.values.shape)
print(X_all_target.shape)

data_for_logit_all = X_all[feature].copy()
data_for_logit_all


(9705, 27)
(9705,)


,sleep_light,sleep_score_latency,sleep_midpoint_time,sleep_duration,sleep_onset_latency,sleep_restless,sleep_score_alignment,sleep_total,sleep_score_deep,sleep_awake
0,11670,94,12390,24480,990,32,99,22620,92,1860
1,11250,99,15750,31200,900,22,100,29340,100,1860
2,14100,97,15270,30420,840,31,100,27210,97,3210
3,15750,91,14520,28980,690,32,100,27090,96,1890
4,18600,72,14940,30180,300,33,100,28980,81,1200
...,...,...,...,...,...,...,...,...,...,...
9700,20580,94,23640,46200,780,42,100,36720,75,9480
9701,18030,99,20340,38280,900,35,100,30000,70,8280
9702,13470,72,13350,26940,300,42,100,24360,95,2580
9703,18420,97,19620,40860,930,39,100,31920,91,8940


In [19]:

scaler_all = MinMaxScaler()
X_all_scaled = scaler_all.fit_transform(data_for_logit_all)

x = torch.tensor(X_all_scaled).float()
y = torch.tensor(X_all_target.values-1).long()

In [ ]:
from collections import defaultdict
 
bs = 32
epoch = 20

for e in range(1,epoch + 1):
    eloss = defaultdict(lambda: [])
    for idx in range(0,len(x),bs):
        opt.zero_grad()
        p = net(x[idx:idx+bs])
        loss = los(p, y[idx:idx+bs])
        loss.backward()
        opt.step()
        # keep track of some summary statistics
        eloss['loss'].append(loss.item())
        eloss['n'].append(x[idx:idx+bs].size(0))
        eloss['corr'].append(sum(p.max(axis=1)[1] == y[idx:idx+bs]))
    print(f'epoch {e:3d} avg loss; {sum(eloss["loss"])/sum(eloss["n"]):.4f}')
    print(f'        acc.;    {sum(eloss["corr"])/sum(eloss["n"]):.4f}')
print("done")

net.eval()
with torch.no_grad():
    final_accuracy = (net(x).argmax(dim=1) == y).sum().item() / len(y)
print(f"Final accuracy: {final_accuracy:.4f}")

epoch   1 avg loss; 0.0182
        acc.;    0.8697
epoch   2 avg loss; 0.0223
        acc.;    0.7733
epoch   3 avg loss; 0.0276
        acc.;    0.6757
epoch   4 avg loss; 0.0271
        acc.;    0.6615
epoch   5 avg loss; 0.0293
        acc.;    0.6284
epoch   6 avg loss; 0.0289
        acc.;    0.6496
epoch   7 avg loss; 0.0267
        acc.;    0.6904
epoch   8 avg loss; 0.0264
        acc.;    0.6752
epoch   9 avg loss; 0.0269
        acc.;    0.6537
epoch  10 avg loss; 0.0289
        acc.;    0.6482
epoch  11 avg loss; 0.0286
        acc.;    0.6252
epoch  12 avg loss; 0.0293
        acc.;    0.6087
epoch  13 avg loss; 0.0274
        acc.;    0.6730
epoch  14 avg loss; 0.0288
        acc.;    0.6640
epoch  15 avg loss; 0.0272
        acc.;    0.6437
epoch  16 avg loss; 0.0295
        acc.;    0.6044
epoch  17 avg loss; 0.0291
        acc.;    0.5669
epoch  18 avg loss; 0.0331
        acc.;    0.5184
epoch  19 avg loss; 0.0315
        acc.;    0.5465
epoch  20 avg loss; 0.0290
    

In [21]:
test_all, test_mean = preprocessing(test, test_label)

test_all_target = test_all['target']

X_test_all = test_all[feature].copy()

test_X_all_scaled = scaler_all.transform(X_test_all)
test_x_all = torch.tensor(test_X_all_scaled).float()
test_y_all = torch.tensor(test_all_target.values - 1).long()

net.eval()

with torch.no_grad():
    predictions = net(test_x_all)
    predicted_labels = predictions.max(dim=1)[1]
    
    accuracy = (predicted_labels == test_y_all).sum().item() / len(test_y_all)
    print(f'Test Accuracy: {accuracy:.4f}')

    test_loss = los(predictions, test_y_all)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Accuracy: 0.1212
Test Loss: 1.1011


성능 향상을 위한 데이터 처리

-> (1) 데이터 불균형 처리
(2) scailing

In [22]:
print(data_for_logit_all.shape)
print(X_all_target.shape)
print(X_test_all.shape)

(9705, 10)
(9705,)
(33, 10)


In [23]:
# OVERSAMPLING
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

# 데이터셋 가져오기
# crossentropyloss를 적용하기 위해서는 label이 0부터 시작해야함
# 현재 target의 label은 1, 2, 3이기 때문에 변환을 해줌.
X = data_for_logit_all.values
y = X_all_target.values - 1

# train, validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# OVERSAMPLING 적용
smote = SMOTE(random_state=42)
X_train_over, y_train_over= smote.fit_resample(X_train, y_train)
print(f'OVERSAMPLING 적용 후 x train dataset: {X_train_over.shape}')

# UNDERSAMPLING 적용
rus = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = rus.fit_resample(X_train, y_train)
print(f'UNDERSAMPLING 적용 후 x train dataset: {X_train_under.shape}')


OVERSAMPLING 적용 후 x train dataset: (12108, 10)
UNDERSAMPLING 적용 후 x train dataset: (1254, 10)


In [31]:
pd.DataFrame(y_train_under).value_counts()

0
0    418
1    418
2    418
Name: count, dtype: int64

In [ ]:
# 모델 학습 - OVERSAMPLING

scaler_over = MinMaxScaler()
X_train = scaler_over.fit_transform(X_train_over)
X_val = scaler_over.transform(X_val)
test_x_all = torch.tensor(scaler_over.transform(X_test_all)).float()

X_train = torch.tensor(X_train_over).float()
y_train = torch.tensor(y_train_over).long()
X_val = torch.tensor(X_val).float()
y_val = torch.tensor(y_val).long()


net_over = DementiaNet()
opt_over = torch.optim.Adam(net_over.parameters(), lr=1e-3)
los_over = nn.CrossEntropyLoss()

batch_over = 64
epoch = 20

for e in range(1,epoch + 1):
    eloss = defaultdict(lambda: [])
    net_over.train()
    for idx in range(0,len(X_train),batch_over):
        opt_over.zero_grad()
        p = net_over(X_train[idx:idx + batch_over])
        loss = los_over(p, y_train[idx:idx + batch_over])
        loss.backward()
        opt_over.step()
        # keep track of some summary statistics
        eloss['loss'].append(loss.item())
        eloss['n'].append(X_train[idx:idx+batch_over].size(0))
        eloss['corr'].append(sum(p.max(axis=1)[1] == y_train[idx:idx + batch_over]))
    train_loss = sum(eloss["loss"]) / len(eloss["n"])
    train_acc = sum(eloss["corr"]) / sum(eloss["n"])
    print(f'Epoch {e:3d} | Train loss: {train_loss:.4f} | Train Accuracy: {train_acc:.4f}')
    
    net_over.eval()
    with torch.no_grad():
        p_val = net_over(X_val)
        val_loss = los_over(p_val, y_val).item()
        val_acc = (p_val.argmax(dim=1) == y_val).sum().item() / len(y_val)
    print(f'          | Validation loss: {val_loss:.4f} | Validation Accuracy: {val_acc:.4f}')
        
print("done")

net_over.eval()
with torch.no_grad():
    final_accuracy = (net_over(X_train).argmax(dim=1) == y_train).sum().item() / len(y_train)
print(f"Final Training Accuracy: {final_accuracy:.4f}")

net_over.eval()

with torch.no_grad():
    predictions = net_over(test_x_all)
    predicted_labels = predictions.max(dim=1)[1]
    
    accuracy = (predicted_labels == test_y_all).sum().item() / len(test_y_all)
    print(f'Test Accuracy: {accuracy:.4f}')

    test_loss = los_over(predictions, test_y_all)
    print(f'Test Loss: {test_loss.item():.4f}')

c:\anaconda3\envs\datamining\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


Epoch   1 | Train loss: 89.2013 | Train Accuracy: 0.7013
          | Validation loss: 1.1900 | Validation Accuracy: 0.0525
Epoch   2 | Train loss: 564.4540 | Train Accuracy: 0.6868
          | Validation loss: 1.2176 | Validation Accuracy: 0.0525
Epoch   3 | Train loss: 799.3688 | Train Accuracy: 0.6818
          | Validation loss: 1.1686 | Validation Accuracy: 0.0525
Epoch   4 | Train loss: 564.7536 | Train Accuracy: 0.6867
          | Validation loss: 1.1326 | Validation Accuracy: 0.0570
Epoch   5 | Train loss: 291.7280 | Train Accuracy: 0.6914
          | Validation loss: 1.1214 | Validation Accuracy: 0.0659
Epoch   6 | Train loss: 277.6242 | Train Accuracy: 0.6895
          | Validation loss: 1.1137 | Validation Accuracy: 0.1233
Epoch   7 | Train loss: 230.7365 | Train Accuracy: 0.6794
          | Validation loss: 1.1167 | Validation Accuracy: 0.1150
Epoch   8 | Train loss: 241.7699 | Train Accuracy: 0.6771
          | Validation loss: 1.0976 | Validation Accuracy: 0.3963
Epoch   9

In [95]:
# 모델 학습 - UNDERSAMPLING
scaler_under = MinMaxScaler()
X_train = scaler_under.fit_transform(X_train_under)
X_val = scaler_under.transform(X_val)
test_x_all = torch.tensor(scaler_under.transform(X_test_all)).float()

X_train = torch.tensor(X_train_under).float()
y_train = torch.tensor(y_train_under).long()
X_val = torch.tensor(X_val).float()
y_val = torch.tensor(y_val).long()

net_under = DementiaNet()
opt_under = torch.optim.Adam(net_under.parameters(), lr=1e-3)
los_under = nn.CrossEntropyLoss()

batch_under = 8
epoch_under = 20

for e in range(1,epoch_under + 1):
    eloss = defaultdict(lambda: [])
    net_under.train()
    for idx in range(0,len(X_train),batch_under):
        opt_under.zero_grad()
        p = net_under(X_train[idx:idx + batch_under])
        loss = los_under(p, y_train[idx:idx + batch_under])
        loss.backward()
        opt_under.step()
        # keep track of some summary statistics
        eloss['loss'].append(loss.item())
        eloss['n'].append(X_train[idx:idx+batch_under].size(0))
        eloss['corr'].append(sum(p.max(axis=1)[1] == y_train[idx:idx + batch_under]))
    train_loss = sum(eloss["loss"]) / len(eloss["n"])
    train_acc = sum(eloss["corr"]) / sum(eloss["n"])
    print(f'Epoch {e:3d} | Train loss: {train_loss:.4f} | Train Accuracy: {train_acc:.4f}')
    
    net_under.eval()
    with torch.no_grad():
        p_val = net_under(X_val)
        val_loss = los_under(p_val, y_val).item()
        val_acc = (p_val.argmax(dim=1) == y_val).sum().item() / len(y_val)
    print(f'          | Validation loss: {val_loss:.4f} | Validation Accuracy: {val_acc:.4f}')
        
print("done")

net_under.eval()
with torch.no_grad():
    final_accuracy = (net_under(X_train).argmax(dim=1) == y_train).sum().item() / len(y_train)
print(f"Final Training Accuracy: {final_accuracy:.4f}")

net_under.eval()

with torch.no_grad():
    predictions = net_under(test_x_all)
    predicted_labels = predictions.max(dim=1)[1]
    
    accuracy = (predicted_labels == test_y_all).sum().item() / len(test_y_all)
    print(f'Test Accuracy: {accuracy:.4f}')

    test_loss = los_under(predictions, test_y_all)
    print(f'Test Loss: {test_loss.item():.4f}')

c:\anaconda3\envs\datamining\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\zz325\AppData\Local\Temp\ipykernel_10152\4190510000.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val = torch.tensor(y_val).long()


Epoch   1 | Train loss: 938.5504 | Train Accuracy: 0.8884
          | Validation loss: 1.2613 | Validation Accuracy: 0.0525
Epoch   2 | Train loss: 4839.4003 | Train Accuracy: 0.7089
          | Validation loss: 1.2001 | Validation Accuracy: 0.0525
Epoch   3 | Train loss: 1860.0803 | Train Accuracy: 0.6954
          | Validation loss: 1.1855 | Validation Accuracy: 0.0525
Epoch   4 | Train loss: 915.4622 | Train Accuracy: 0.6874
          | Validation loss: 1.1694 | Validation Accuracy: 0.0525
Epoch   5 | Train loss: 424.1191 | Train Accuracy: 0.6914
          | Validation loss: 1.1588 | Validation Accuracy: 0.0525
Epoch   6 | Train loss: 318.1759 | Train Accuracy: 0.7129
          | Validation loss: 1.1525 | Validation Accuracy: 0.0525
Epoch   7 | Train loss: 126.4437 | Train Accuracy: 0.6866
          | Validation loss: 1.1485 | Validation Accuracy: 0.0525
Epoch   8 | Train loss: 59.3321 | Train Accuracy: 0.6762
          | Validation loss: 1.1394 | Validation Accuracy: 0.0525
Epoch  